# Hosts: Asset trait introspection


## Setup

See "Hello OpenAssetIO" notebook for details on how to bootstrap OpenAssetIO. 

In [55]:
from resources import helpers

manager, context = helpers.bootstrap("resources/querying_entity_traits/openassetio_config.toml")

## Getting started

In the following examples we're going to ask the manager about the traits associated with an asset.

We've been given a URI by a colleague, which we need to turn into an `EntityReference` before we can use it to query the asset management system.

In [56]:
logo_ref = manager.createEntityReference("bal:///project_artwork/logos/openassetio")

helpers.display_result(repr(logo_ref))

> **Result:**
> `<openassetio.EntityReference bal:///project_artwork/logos/openassetio>`

Now we have an entity reference for our logo, we can use the API to learn more about it.

## The `entityTraits` method

The `entityTraits` methods allows a host to query the manager for the traits associated with a given entity reference. It has two access modes. `kRead` mode will return all the traits that the manager associates with an entity. `kWrite` mode returns the minimal set of traits required to publish to the given entity reference.

The `entityTraits` method is a required method that managers _must_ implement, so there is no need to perform a `hasCapability` check before using it.

The following examples illustrate usage of the `entityTraits` method.

## Querying the full set of traits associated with an entity

OpenAssetIO does not try to fully define entity types. What it does define are traits that an entity satisfies. An entity is theoretically fully specified by some (large) subset of traits in the universe of all possible traits. A manager understands some subset of that theoretical complete subset of traits. This is the "trait set" of the entity, with respect to a given manager.

In order to find out what the trait set of an entity is, we can use the `entityTraits` API method.

In [57]:
from openassetio.access import EntityTraitsAccess

entity_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kRead, context)

helpers.display_result(entity_trait_set)

> **Result:**
> `{'openassetio-mediacreation:identity.DisplayName', 'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:timeDomain.FrameRanged', 'openassetio-mediacreation:lifecycle.Version', 'openassetio-mediacreation:usage.Entity'}`

`EntityTraitAccess.kRead` tells the manager the reason why you want the trait set. In this case we want to use the result in a read context, e.g. `resolve`ing properties, grouping entities of the same type in a UI, or determining the type of entity after a drag-and-drop of an entity reference into the application.

What happens when we `resolve` this trait set?

In [58]:
from openassetio.access import ResolveAccess

entity_data = manager.resolve(logo_ref, entity_trait_set, ResolveAccess.kRead, context)

helpers.display_result(entity_data)

> **Result:**
> `TraitsData({"openassetio-mediacreation:identity.DisplayName", "openassetio-mediacreation:content.LocatableContent", "openassetio-mediacreation:lifecycle.Version"})`

Note that not all traits have `resolve`able properties - many are simply used to aid classification. However, it is safe to `resolve` the full trait set - those that do not have properties are simply missing from the response.

As an aside, we can filter the trait set through `managementPolicy`, if we wish to know in advance which traits any given manager is capable of providing data for

In [59]:
from openassetio.access import PolicyAccess

[policy_data] = manager.managementPolicy([entity_trait_set], PolicyAccess.kRead, context)

helpers.display_result(policy_data)

> **Result:**
> `TraitsData({"openassetio-mediacreation:managementPolicy.Managed", "openassetio-mediacreation:lifecycle.Version", "openassetio-mediacreation:content.LocatableContent", "openassetio-mediacreation:identity.DisplayName"})`

Note how the trait set returned from `managementPolicy` (minus any `managementPolicy` traits) matches the trait set we successfully `resolve`d.  This is missing the `FrameRangedTrait`, despite it being included in the result of `entityTraits`, meaning that the `FrameRangedTrait` is a quality of the entity, but properties of it cannot be resolved (perhaps due to some technical limitation of the manager).

In this way `managementPolicy` can be used to filter the trait set of an entity to only those traits that have `resolve`able properties. However, note that `mangementPolicy` does not take an entity reference argument, only trait set(s). The result of `managementPolicy` is therefore constant for any given manager, regardless of entity.

In practice, a well-behaved host will cherry-pick only those traits the host needs to `resolve`. That trait set should then be passed to `managementPolicy` to determine what the manager can actually provide. This process is independent of `entityTraits`. That is, the host should know at application startup which traits it's going to need, and so query `managementPolicy` ahead of time and store the result for use later. See relevant documentation for `managementPolicy` for more information.

## Querying the required set of traits for publishing an entity

By using the `kWrite` access mode of `entityTraits`, we can query the minimal trait set that _must_ be provided when publishing to a particular entity reference.

In most applications it is expected that the user will provide (via previous interaction with the manager) an entity reference that is suitable for a particular use-case. With this assumption, the host should not need to include additional data that is irrelevant for the use-case when publishing to an entity reference. That is, the host should not need to `resolve` data for a given entity reference for the sole purpose of immediately giving the data back again to the manager. 

This corresponds to an important philosophy of OpenAssetIO: when the user provides an entity reference, the host should not make assumptions about its provenance. It may be a reference to an existing entity, or a reference to a container, or a placeholder reference to an entity that doesn't exist yet, etc.

As such, `entityTraits` should rarely appear in publishing code. However, there are use cases where it is valuable.

### An empty trait set is a valid response

BAL has no restrictions on publishing to new entity references. So what happens if we just make up an entity reference, and try to get the trait set for it?

In [60]:
new_entity_ref = manager.createEntityReference("bal:///some/new/ref")

entity_trait_set = manager.entityTraits(new_entity_ref, EntityTraitsAccess.kWrite, context)

helpers.display_result(entity_trait_set)

> **Result:**
> `set()`

...an empty trait set is returned.

If the manager does not know the trait set, or it's not applicable, the returned trait set may be empty. For example, as illustrated here, the entity reference may point to a new entity with no type constraints. 

### Deciding whether an entity reference is appropriate

Let's say we are in a widget used for publishing 3D models. The user provides an entity reference they want to publish to. 

Since we deal with 3D models, we expect this entity reference to support publishing a trait set of `{GeometryTrait.kId, LocatableContentTrait.kId}`. But the user gives us `logo_ref`...

In [61]:
from openassetio_mediacreation.traits.content import LocatableContentTrait
from openassetio_mediacreation.traits.threeDimensional import GeometryTrait

entity_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kWrite, context)

helpers.display_result(entity_trait_set)

# Check if the minimal trait set required for publishing is satisfied by the trait set we're planning to publish.
is_entity_supported = entity_trait_set <= {GeometryTrait.kId, LocatableContentTrait.kId}

helpers.display_result(f"Does our widget support this entity? {is_entity_supported}")

> **Result:**
> `{'openassetio-mediacreation:identity.DisplayName', 'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:usage.Entity', 'openassetio-mediacreation:timeDomain.FrameRanged'}`

> **Result:**
> `Does our widget support this entity? False`

As expected, the trait set of the logo is not contained within our expected trait set, so our widget could flag an error to the user at this point.

A host could skip this check and try to publish regardless, reporting any error (from `preflight` or `register`) to the user. The `preflight` method is intended, in part, as a validation step, so this workflow is not unreasonable. However, doing this would likely provide a worse UX, since the widget could not be pre-emptively disabled.

#### Aside: the minimal trait set required for publishing 

In the previous example we see that the resulting trait set is smaller than that for `kRead`. This is because it is the minimal set of traits that the new entity _must_ possess when publishing to this entity reference.

Therefore, this trait set _must_ be imbued in the `TraitsData` given to `preflight` and `register` during the publishing process.

When registering new data, the full trait set defines what 'kind' of entity is being published, regardless of the specifics of what actual data is provided.

As a consequence, the minimal trait set _does not_ indicate whether a manager is capable of storing or dictating their properties (e.g. providing a file path to save to). See relevant documentation for `managementPolicy` for more information on how to determine that.

Again, many of these traits will not have properties associated with them at all. These must still be passed to `preflight`/`register` so that the manager knows the kind of entity you are publishing.

So what happens if we forget to include all these traits?

In [62]:
from openassetio.trait import TraitsData
from openassetio.access import PublishingAccess
from openassetio_mediacreation.traits.auth import BearerTokenTrait

data = TraitsData()
LocatableContentTrait(data).setLocation("file:///some/path")
BearerTokenTrait(data).setToken("==ZxErn43G")

entity_ref_or_error = manager.preflight(
    logo_ref, data, PublishingAccess.kWrite, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(entity_ref_or_error)

> **Result:**
> `bal:///project_artwork/logos/openassetio`

We get an `InvalidTraitSet` error if the provided trait set is not compatible, or sufficiently complete, for publishing to the entity reference.

TODO(DF): There is also `InvalidPreflightHint`, but this doesn't obviously apply to `register`. We could use that instead, or rename it, or come up with a new `BatchElementError` code.

## Errors when querying the trait set

### Entities that don't exist (yet)

For `kRead` access, the entity must exist:

In [63]:
future_ref = manager.createEntityReference("bal:///project_artwork/logos/new")

trait_set_or_error = manager.entityTraits(
    logo_ref, EntityTraitsAccess.kRead, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(trait_set_or_error)

> **Result:**
> `{'openassetio-mediacreation:identity.DisplayName', 'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:timeDomain.FrameRanged', 'openassetio-mediacreation:lifecycle.Version', 'openassetio-mediacreation:usage.Entity'}`

If the entity doesn't exist we get an `EntityAccessError`.

On the other hand, this entity reference is fine for `kWrite`:

In [64]:
trait_set_or_error = manager.entityTraits(
    future_ref, EntityTraitsAccess.kWrite, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(trait_set_or_error)

> **Result:**
> `set()`

Here, once again, we get the minimal trait set required for publishing to this entity reference.

### Read-only entities

BAL doesn't support publishing to a specific version tag (i.e. doesn't support overwriting a version). 

Let's see what happens when we try to get the traits of a specific version, but we use `kWrite` access, indicating that we intend to publish to the entity reference.

In [65]:
v1_ref = manager.createEntityReference("bal:///project_artwork/logos/openassetio?v=1")

trait_set_or_error = manager.entityTraits(
    logo_ref, EntityTraitsAccess.kWrite, context, manager.BatchElementErrorPolicyTag.kVariant)

helpers.display_result(trait_set_or_error)

> **Result:**
> `{'openassetio-mediacreation:identity.DisplayName', 'openassetio-mediacreation:content.LocatableContent', 'openassetio-mediacreation:usage.Entity', 'openassetio-mediacreation:timeDomain.FrameRanged'}`

Since the entity is read-only, we get an `EntityAccessError`.

## Generic resolve and re-publish

A rare use-case is for a tool that blindly resolves, updates, and re-publishes an entity, regardless what type of entity it is. 

Such generic re-publishing is discouraged and dangerous. For example, if "approval" status is blindly resolved and re-published, the manager may not know how it should handle this properly. A better approach to such a generic tool is to present the traits and their properties to the user to select before re-publishing.

However, such workflows are possible and have their place in a pipeline. This can be accomplished by making use of `entityTraits`. 

### Re-publishing a mutated entity

The following presents an example of blindly updating the display name for any entity. Note that we make use of the fact that managers should silently ignore data that it cannot persist when publishing.

In [66]:
from openassetio_mediacreation.traits.identity import DisplayNameTrait

# Get the complete trait set of the entity.
entity_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kRead, context)

# Ensure the manager will accept a publish of this entity with an updated display name.
[policy_data] = manager.managementPolicy(
    [entity_trait_set | {DisplayNameTrait.kId}], PolicyAccess.kWrite, context)

if not DisplayNameTrait.kId in policy_data.traitSet():
    raise Exception("Cannot update display name of this entity")

# Get all the properties of the given entity.
data_to_publish = manager.resolve(logo_ref, entity_trait_set, ResolveAccess.kRead, context)

# Any traits without properties, or where the manager cannot provide them, will be missing from the data.
# We still need to imbue those traits, so that manager knows what kind of entity we are publishing.
data_to_publish.addTraits(entity_trait_set)

# Create/update the name of the entity.
DisplayNameTrait(data_to_publish).setName("My New Name")

# Publish it. Any properties we `resolve`d that cannot be re-published will be silently ignored.
updated_ref = manager.register(logo_ref, data_to_publish, PublishingAccess.kWrite, context)

### Re-publishing a minimal entity

The following example dives deeper into the interaction between `entityTraits`, `managementPolicy` and `resolve` for a generic re-publisher. 

We want to re-publish a minimal entity (i.e. only the traits absolutely required for the given entity reference), with two new/updated traits. One of the traits, `BearerTokenTrait`, might not be supported, and the other trait, `LocatableContentTrait`, might have its properties dictated by the manager.

In [67]:
# The minimum set of traits required to publish to this entity reference.
minimum_trait_set = manager.entityTraits(logo_ref, EntityTraitsAccess.kWrite, context)

# Whatever the minimum trait set is, we know we want to publish a location and auth token.
desired_trait_set = minimum_trait_set | {BearerTokenTrait.kId, LocatableContentTrait.kId}

# Get the set of traits that have properties the manager can persist.
[policy_for_desired_traits] = manager.managementPolicy(
    [desired_trait_set], PolicyAccess.kWrite, context)

# Filter down the desired traits to only those that are supported.
trait_set_to_publish = desired_trait_set & policy_for_desired_traits.traitSet()

# We want to keep (the minimum amount of) data from the previous version, except for the values we're going to
# provide.
trait_set_to_keep = trait_set_to_publish - {BearerTokenTrait.kId, LocatableContentTrait.kId}

# Get the properties that we wish to keep from the current version.
data_to_publish = manager.resolve(logo_ref, trait_set_to_keep, ResolveAccess.kManagerDriven, context)

# Any traits without properties, or where the manager cannot provide them, will be missing from the data.
# We still need to imbue those traits, so that manager knows what kind of entity we are publishing.
data_to_publish.addTraits(minimum_trait_set)

# Get the manager's policy for dictating trait properties, i.e. which traits the manager can `kDerive` for us.
[policy_for_derived_traits] = manager.managementPolicy(
    [trait_set_to_publish], PolicyAccess.kManagerDriven, context)

# Check if the manager can derive a location for us.
if LocatableContentTrait.kId in policy_for_derived_traits.traitSet():
    # Imbue an empty LocatableContentTrait, so that the manager is aware in `preflight` that we intend to publish
    # this trait. We will ask the manager to fill in the value for us before calling `register`.
    LocatableContentTrait.imbueTo(data_to_publish)
else:
    # If the manager doesn't want to provide a location for entities of this type, use a default.
    LocatableContentTrait(data_to_publish).setLocation("file:///tmp/file")

# Manager might not support BearerTokenTrait.
if BearerTokenTrait.kId in trait_set_to_publish:
    # BearerTokenTrait is supported, so imbue and configure.
    BearerTokenTrait(data_to_publish).setToken("==ZxErn43G")

# We can now successfully begin the publishing process.
working_ref = manager.preflight(logo_ref, data_to_publish, PublishingAccess.kWrite, context)

# Check if the manager can provide a location to us.
if LocatableContentTrait.kId in policy_for_derived_traits.traitSet():
    derived_data = manager.resolve(
        working_ref, {LocatableContentTrait.kId}, ResolveAccess.kManagerDriven, context)

    # TODO(DF): `upsert` function for `TraitsData`.
    LocatableContentTrait(data_to_publish).setLocation(
        LocatableContentTrait(derived_data).getLocation())
    
# [Do some work to write the new file...]

# We can now finally publish
updated_ref = manager.register(logo_ref, data_to_publish, PublishingAccess.kWrite, context)

BatchElementException: entityAccessError: Unsupported access mode for resolve [index=0] [access=managerDriven] [entity=bal:///project_artwork/logos/openassetio]